In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import itertools
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

import sys
sys.path.append('../')
import utils
import settings

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

In [ ]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = utils.read_files(diff_path='../')
X_reshaped_train_estimated_a, X_reshaped_train_estimated_b, X_reshaped_train_estimated_c, X_reshaped_train_observed_a, X_reshaped_train_observed_b, X_reshaped_train_observed_c, X_reshaped_test_estimated_a, X_reshaped_test_estimated_b, X_reshaped_test_estimated_c = utils.get_reshaped_files(diff_path='../')

In [ ]:
train_a_copy = train_a.copy()

In [ ]:
train_a_copy.set_index('time', inplace=True)
max_values_per_day = train_a_copy.resample('D').max()
max_values_per_day

In [ ]:
train_a_daily = train_a[train_a['time'].dt.strftime('%H:%M:%S').str.endswith('12:00:00')]
train_a_daily

In [ ]:
# train_a_daily_max = train_a[]

In [ ]:
plt.plot(train_a_daily['pv_measurement'])

In [ ]:
plt.plot(max_values_per_day['pv_measurement'])

In [ ]:
train_a = train_a.rename(columns={'time': 'ds', 'pv_measurement': 'y'})[:29667] # 29667
train_a["y"] = (train_a["y"] - train_a["y"].mean()) / train_a["y"].std()
# train_a["y"] /= train_a["y"].max()

In [ ]:
train_a_daily = train_a_daily.rename(columns={'time': 'ds', 'pv_measurement': 'y'})[:29667] # 29667
train_a_daily["y"] = (train_a_daily["y"] - train_a_daily["y"].mean()) / train_a_daily["y"].std()
# train_a["y"] /= train_a["y"].max()

In [ ]:
max_values_per_day = max_values_per_day.reset_index()

In [ ]:
max_values_per_day = max_values_per_day.rename(columns={'time': 'ds', 'pv_measurement': 'y'})[:29667] # 29667
max_values_per_day["y"] = (max_values_per_day["y"] - max_values_per_day["y"].mean()) / max_values_per_day["y"].std()
max_values_per_day

In [ ]:
m = Prophet(changepoint_prior_scale=0.05)
m.fit(train_a)

In [ ]:
m = Prophet(changepoint_prior_scale=0.05)
m.fit(train_a_daily)

In [ ]:
# future = m.make_future_dataframe(periods=66 * 24, freq='D')
future = m.make_future_dataframe(periods=66, freq='D')
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['yhat']].tail()

In [ ]:
fig1 = m.plot(forecast) 

In [ ]:
forecast[forecast['ds'] >= '2023-05-01'][['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast['yhat']

In [ ]:
plt.plot(forecast['yhat'])

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(train_a["ds"], train_a["y"])

In [ ]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here
df = max_values_per_day
print(all_params)
# Use cross validation to evaluate all parameters
for params in all_params:
    print('params:', params)
    m = Prophet(**params).fit(df)  # Fit model with given params
    df_cv = cross_validation(m, horizon='30 days', parallel="processes")
    # df_cv = cross_validation(m, cutoffs=cutoffs, horizon='30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

In [ ]:
m = Prophet(**best_params)
m.fit(max_values_per_day)
future = m.make_future_dataframe(periods=66, freq='H')
fig1 = m.plot(forecast)